In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras import backend as K
import matplotlib.pyplot as plt
import pandas as pd
import datetime

In [2]:
path = '/home/ege/Repo/SideChannel-AdversarialAI/Tensorflow/TwoClassModelsV2'

In [13]:
accuracy_df = pd.read_csv(path+"/Accuracy.csv")
accuracy_df


,Class Tuples,A_Count,B_Count,Other_Count,Correct?
0,"0,3",110,130,0,240
1,"0,4",97,143,0,240
2,"2,4",0,240,0,240
3,"3,0",135,105,0,240
4,"3,4",119,121,0,240
...,...,...,...,...,...
177,"2,9",0,0,240,-240
178,"2,10",0,0,240,-240
179,"7,2",0,0,240,-240
180,"10,1",0,0,240,-240


In [23]:
for index, row in accuracy_df.iterrows():
    
    
    
    my_list = row['Class Tuples'].split(",")
    classA = my_list[0]
    classB = my_list[1]
    
    points = pd.read_csv(path+"/"+str(classA)+"/"+str(classB)+"/points.csv")
    points = points.drop(['Unnamed: 0'], axis=1)    
    #points.dropna(inplace = True) 
    X_A = points["X_A"].tolist()
    Y_A = points["Y_A"].tolist()
    
    X_B = points["X_B"].tolist()
    Y_B = points["Y_B"].tolist()

    print(X_A)
    
    
    #print(str(my_list[0])+","+str(my_list[1]))

[nan, 0.8004847764968872, 0.1972982138395309, -0.1085446327924728, 0.3792687058448791, 0.005992267280817, -0.7574417591094971, -1.169319748878479, -0.1827031373977661, 0.076981134712696, 0.7548478841781616, 0.3399649262428283, -0.4515566825866699, 0.3788616359233856, 1.2165460586547852, -1.9287394285202024, -0.1033211275935173, -0.908470630645752, -0.0924483314156532, nan, -0.0541065335273742, 0.3944398164749145, 0.339043378829956, 1.2544137239456177, -1.8719843626022337, -1.5499517917633057, 0.2202949523925781, -0.4636628031730652, 1.7959964275360107, -0.8276326656341553, 0.5751961469650269, 1.9175424575805664, -0.0137027651071548, -1.8525855541229248, -0.6836068630218506, -1.0867434740066528, -0.8323206901550293, 0.3652077913284302, -1.32487154006958, 0.5381320118904114, 1.4498730897903442, -0.1503917872905731, 0.4266641438007355, 0.1000333726406097, -1.379255294799805, -0.0277956966310739, 0.7854259014129639, -1.4975961446762085, 0.0841977000236511, 0.8296213150024414, -0.0859314575